In [25]:
import dask.dataframe as dd
import numpy as np
import pandas as pd

In [5]:
ratings = dd.read_table("ml-100k/u.data",names = ["userID","movieID","rating","_"],usecols = ["userID","movieID","rating"],
                    dtype = {"rating":np.float64}).set_index("userID")

In [6]:
ratings.head()

,movieID,rating
userID,,
1,55,5.0
1,203,4.0
1,183,5.0
1,150,5.0
1,68,4.0


In [7]:
joinedRatings = ratings.join(ratings,lsuffix='_1', rsuffix='_2')
joinedRatings.head()

,movieID_1,rating_1,movieID_2,rating_2
userID,,,,
1,55,5.0,55,5.0
1,55,5.0,203,4.0
1,55,5.0,183,5.0
1,55,5.0,150,5.0
1,55,5.0,68,4.0


In [8]:
joinedRatings.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 4 entries, movieID_1 to rating_2
dtypes: float64(2), int64(2)

In [9]:
uniqueJoinedRatings = joinedRatings.query("movieID_1 < movieID_2")
uniqueJoinedRatings.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 4 entries, movieID_1 to rating_2
dtypes: float64(2), int64(2)

In [11]:
moviePairs = uniqueJoinedRatings.set_index("movieID_1")
moviePairs.head()
moviePairs.shape

(Delayed('int-9b573ea9-3dd2-47ab-b083-5368291e4bae'), 3)

In [12]:
moviePairRatings = moviePairs.groupby(["movieID_1","movieID_2"])

In [13]:
moviePairRatings.count().info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 2 entries, rating_1 to rating_2
dtypes: int64(2)

In [14]:
moviePairRatings.size().shape

(dd.Scalar<size-ag..., dtype=int32>,)

In [ ]:
import random
def chivato(x):
    l =random.randint(0,300000)
    if l == 0:
        print(x)
    return pd.Series({"a":2,"b":3.0})

In [13]:
moviePairRatings.apply(chivato)

                     rating_1  rating_2
movieID_1 movieID_2                    
169       1091            4.0       2.0
          1091            5.0       4.0
          1091            5.0       2.0
          1091            5.0       4.0
          1091            3.0       2.0
          1091            5.0       3.0
          1091            3.0       4.0
          1091            5.0       1.0
          1091            3.0       2.0
          1091            1.0       1.0
          1091            3.0       3.0
          1091            5.0       4.0
          1091            4.0       2.0
          1091            4.0       1.0
                     rating_1  rating_2
movieID_1 movieID_2                    
240       1355            1.0       1.0
          1355            3.0       1.0
          1355            4.0       4.0
                     rating_1  rating_2
movieID_1 movieID_2                    
273       1183            3.0       3.0
          1183            5.0       1.0


a    b
movieID_1 movieID_2          
1         2          2.0  3.0
          3          2.0  3.0
          4          2.0  3.0
          5          2.0  3.0
          6          2.0  3.0
          7          2.0  3.0
          8          2.0  3.0
          9          2.0  3.0
          10         2.0  3.0
          11         2.0  3.0
          12         2.0  3.0
          13         2.0  3.0
          14         2.0  3.0
          15         2.0  3.0
          16         2.0  3.0
          17         2.0  3.0
          18         2.0  3.0
          19         2.0  3.0
          20         2.0  3.0
          21         2.0  3.0
          22         2.0  3.0
          23         2.0  3.0
          24         2.0  3.0
          25         2.0  3.0
          26         2.0  3.0
          27         2.0  3.0
          28         2.0  3.0
          29         2.0  3.0
          30         2.0  3.0
          31         2.0  3.0
...                  ...  ...
1663      1667       2.0  3.0
          1668       2.0  3.0
          1669       2.0  3.0
          1670       2.0  3.0
1664      1665       2.0  3.0
          1666       2.0  3.0
          1667       2.0  3.0
          1668       2.0  3.0
          1669       2.0  3.0
          1670       2.0  3.0
1665      1666       2.0  3.0
          1667       2.0  3.0
          1668       2.0  3.0
          1669       2.0  3.0
          1670       2.0  3.0
1666      1667       2.0  3.0
          1668       2.0  3.0
          1669       2.0  3.0
          1670       2.0  3.0
1667      1668       2.0  3.0
          1669       2.0  3.0
          1670       2.0  3.0
1668      1669       2.0  3.0
          1670       2.0  3.0
1669      1670       2.0  3.0
1672      1681       2.0  3.0
1675      1676       2.0  3.0
1678      1679       2.0  3.0
          1680       2.0  3.0
1679      1680       2.0  3.0

[983206 rows x 2 columns]

In [26]:
from math import sqrt
def computeCosineSimilarity(ratingPairs):
    x = ratingPairs["rating_1"]
    y = ratingPairs["rating_2"]
    
    #sum_xx = x @ x.T
    #sum_yy = y @ y.T
    #sum_xy = x @ y.T
    sum_xx = x.dot(x.T)
    sum_yy = y.dot(y.T)
    sum_xy = x.dot(y.T)
    
    numerator = sum_xy
    denominator = sqrt(sum_xx) * sqrt(sum_yy)

    score = 0
    if (denominator):
        score = (numerator / (float(denominator)))

    return pd.Series({"score":score, "numPairs":ratingPairs.shape[0]})

In [27]:
moviePairSimilarities = moviePairRatings.apply(computeCosineSimilarity, meta={'score': 'f8', 'numPairs': 'i4'})

moviePairSimilarities.compute().head()

score  numPairs
movieID_1 movieID_2                    
1         2          0.948737     104.0
          3          0.913300      78.0
          4          0.942907     149.0
          5          0.961364      57.0
          6          0.955119      14.0

In [19]:
movieNames = pd.read_table("ml-100k/u.item",names = ["movieID","title"],usecols = ["movieID","title"],
                    sep ="|",index_col = "movieID", encoding = "cp1252")
movieNames.head()


,title
movieID,
1,Toy Story (1995)
2,GoldenEye (1995)
3,Four Rooms (1995)
4,Get Shorty (1995)
5,Copycat (1995)


In [20]:
movieID = 50
scoreThreshold = 0.97
coOccurenceThreshold = 50


In [18]:
filteredResults = moviePairSimilarities.query(
    "((movieID_1 == @movieID) or (movieID_1 == @movieID)) and (score>@scoreThreshold) and (numPairs>@coOccurenceThreshold)"
    ).sort_values(by="score",ascending=False) 
filteredResults

ValueError: Metadata inference failed in `query`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
UndefinedVariableError("name 'movieID_1' is not defined")

Traceback:
---------
  File "C:\anaconda3\lib\site-packages\dask\dataframe\utils.py", line 137, in raise_on_meta_error
    yield
  File "C:\anaconda3\lib\site-packages\dask\dataframe\core.py", line 3583, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
  File "C:\anaconda3\lib\site-packages\dask\utils.py", line 688, in __call__
    return getattr(obj, self.method)(*args, **kwargs)
  File "C:\anaconda3\lib\site-packages\pandas\core\frame.py", line 2847, in query
    res = self.eval(expr, **kwargs)
  File "C:\anaconda3\lib\site-packages\pandas\core\frame.py"

In [22]:
for i,v in filteredResults.iterrows():
    if i[0] == movieID:
        recommended = i[1]
    else:
        recommended = i[0]
    nR = movieNames.loc[recommended,"title"]
    print(nR,v["score"])

Empire Strikes Back, The (1980) 0.989552207839
Return of the Jedi (1983) 0.985723086125
Raiders of the Lost Ark (1981) 0.981760098873
20,000 Leagues Under the Sea (1954) 0.97893856055
12 Angry Men (1957) 0.977657612045
Close Shave, A (1995) 0.977594829105
African Queen, The (1951) 0.976469222267
Sting, The (1973) 0.975151293774
Wrong Trousers, The (1993) 0.974868135546
Wallace & Gromit: The Best of Aardman Animation (1996) 0.97418161283
Indiana Jones and the Last Crusade (1989) 0.973539482999
North by Northwest (1959) 0.973453431527
Philadelphia Story, The (1940) 0.973429461163
Bridge on the River Kwai, The (1957) 0.972759163953
Casablanca (1942) 0.972657062373
L.A. Confidential (1997) 0.972507158872
Around the World in 80 Days (1956) 0.972453952687
Right Stuff, The (1983) 0.971696679601
Glory (1989) 0.971672803535
Princess Bride, The (1987) 0.971387496344
Pinocchio (1940) 0.971249981891
Good Will Hunting (1997) 0.971057126565
Vertigo (1958) 0.970218559546


In [1]:
moviePairSimilarities.to_cvs("similarities.json")

NameError: name 'moviePairSimilarities' is not defined